In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1209133,2021-04-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1209134,2021-04-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1209135,2021-04-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1209136,2021-04-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32335,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32337,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32339,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32341,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32343,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
744458,2021-04-23,Arkansas,Arkansas,5001,2089,36.00,5000,Arkansas,AR,Arkansas,State,3017804
744460,2021-04-24,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804
744462,2021-04-25,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804
744464,2021-04-26,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1209133,2021-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1209134,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1209135,2021-04-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1209136,2021-04-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-27') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
462,2021-04-27,Snohomish,Washington,35482,564.00,53061,WA,County,822083,4316.11,68.61
922,2021-04-27,Cook,Illinois,530975,10466.00,17031,IL,County,5150233,10309.73,203.21
1381,2021-04-27,Orange,California,269838,4939.00,6059,CA,County,3175692,8496.98,155.53
1839,2021-04-27,Maricopa,Arizona,535436,9840.00,4013,AZ,County,4485414,11937.27,219.38
2297,2021-04-27,Los Angeles,California,1232742,23802.00,6037,CA,County,10039107,12279.40,237.09
...,...,...,...,...,...,...,...,...,...,...,...
1208473,2021-04-27,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1208670,2021-04-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1208836,2021-04-27,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1208998,2021-04-27,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
462,2021-04-27,Snohomish,Washington,35482,564.00,53061,WA,County,822083,4316.11,68.61,68.61,4316.11
922,2021-04-27,Cook,Illinois,530975,10466.00,17031,IL,County,5150233,10309.73,203.21,203.21,10309.73
1381,2021-04-27,Orange,California,269838,4939.00,6059,CA,County,3175692,8496.98,155.53,155.53,8496.98
1839,2021-04-27,Maricopa,Arizona,535436,9840.00,4013,AZ,County,4485414,11937.27,219.38,219.38,11937.27
2297,2021-04-27,Los Angeles,California,1232742,23802.00,6037,CA,County,10039107,12279.40,237.09,237.09,12279.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208473,2021-04-27,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1208670,2021-04-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1208836,2021-04-27,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1208998,2021-04-27,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
778433,2021-04-27,Hale,Alabama,2202,75.00,1065,AL,County,14651,15029.69,511.91,511.91,15029.69,1
847303,2021-04-27,Clarke,Alabama,3499,60.00,1025,AL,County,23622,14812.46,254.00,254.00,14812.46,2
609081,2021-04-27,Lowndes,Alabama,1383,53.00,1085,AL,County,9726,14219.62,544.93,544.93,14219.62,3
453186,2021-04-27,Franklin,Alabama,4226,82.00,1059,AL,County,31362,13474.91,261.46,261.46,13474.91,4
540060,2021-04-27,Etowah,Alabama,13739,352.00,1055,AL,County,102268,13434.31,344.19,344.19,13434.31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146417,2021-04-27,Platte,Wyoming,621,11.00,56031,WY,County,8393,7399.02,131.06,131.06,7399.02,19
742885,2021-04-27,Converse,Wyoming,991,17.00,56009,WY,County,13822,7169.73,122.99,122.99,7169.73,20
923550,2021-04-27,Lincoln,Wyoming,1392,12.00,56023,WY,County,19830,7019.67,60.51,60.51,7019.67,21
1027856,2021-04-27,Niobrara,Wyoming,151,2.00,56027,WY,County,2356,6409.17,84.89,84.89,6409.17,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
609081,2021-04-27,Lowndes,Alabama,1383,53.00,1085,AL,County,9726,14219.62,544.93,544.93,14219.62,3,1
778433,2021-04-27,Hale,Alabama,2202,75.00,1065,AL,County,14651,15029.69,511.91,511.91,15029.69,1,2
259063,2021-04-27,Walker,Alabama,7184,278.00,1127,AL,County,63521,11309.65,437.65,437.65,11309.65,28,3
538464,2021-04-27,Clay,Alabama,1520,56.00,1027,AL,County,13235,11484.70,423.12,423.12,11484.70,23,4
656838,2021-04-27,Greene,Alabama,920,34.00,1063,AL,County,8111,11342.62,419.18,419.18,11342.62,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655249,2021-04-27,Hot Springs,Wyoming,366,3.00,56017,WY,County,4413,8293.68,67.98,67.98,8293.68,17,19
923550,2021-04-27,Lincoln,Wyoming,1392,12.00,56023,WY,County,19830,7019.67,60.51,60.51,7019.67,21,20
900540,2021-04-27,Uinta,Wyoming,2173,12.00,56041,WY,County,20226,10743.60,59.33,59.33,10743.60,5,21
257848,2021-04-27,Teton,Wyoming,3725,9.00,56039,WY,County,23464,15875.38,38.36,38.36,15875.38,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4316.11,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4316.11,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4316.11,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4316.11,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4316.11,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206298,2021-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1206299,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1206300,2021-04-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1206301,2021-04-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
775205,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15029.69,2,1,1.00
775206,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15029.69,2,1,0.00
775207,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15029.69,2,1,0.00
775208,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15029.69,2,1,0.00
775209,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,15029.69,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938994,2021-04-23,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
938995,2021-04-24,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
938996,2021-04-25,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
938997,2021-04-26,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5656.65,6,23,1.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
606641,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,1.00,0.00
606642,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
606643,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
606644,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
606645,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604646,2021-04-23,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10817.90,23,4,14.00,0.00
604647,2021-04-24,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10817.90,23,4,0.00,0.00
604648,2021-04-25,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10817.90,23,4,0.00,0.00
604649,2021-04-26,Albany,Wyoming,4198,11.00,56001,WY,County,38880,10797.33,28.29,28.29,10817.90,23,4,14.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-27') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
309251,2021-04-27,Imperial,California,28148,719.00,6025,CA,County,181215,15532.93,396.77,396.77,15532.93,1,2,12.00,0.00
2297,2021-04-27,Los Angeles,California,1232742,23802.00,6037,CA,County,10039107,12279.40,237.09,237.09,12279.40,2,5,273.00,3.00
153999,2021-04-27,San Bernardino,California,295665,4572.00,6071,CA,County,2180085,13562.09,209.72,209.72,13562.09,3,4,100.00,1.00
541608,2021-04-27,Inyo,California,1396,36.00,6027,CA,County,18039,7738.79,199.57,199.57,7738.79,4,26,0.00,0.00
68869,2021-04-27,Stanislaus,California,61095,1040.00,6099,CA,County,550660,11094.87,188.86,188.86,11094.87,5,9,70.00,1.00
31917,2021-04-27,Riverside,California,298259,4563.00,6065,CA,County,2470546,12072.59,184.70,184.70,12072.59,6,6,258.00,4.00
85386,2021-04-27,Tulare,California,49547,834.00,6107,CA,County,466195,10627.96,178.90,178.90,10627.96,7,10,32.00,1.00
54795,2021-04-27,San Joaquin,California,72281,1346.00,6077,CA,County,762148,9483.85,176.61,176.61,9483.85,8,19,294.00,0.00
31083,2021-04-27,Fresno,California,101099,1662.00,6019,CA,County,999101,10119.00,166.35,166.35,10119.00,9,14,38.00,0.00
490825,2021-04-27,Merced,California,31489,454.00,6047,CA,County,277680,11340.03,163.50,163.50,11340.03,10,8,37.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1145968,2021-04-27,Lassen,California,5685,1.00,6035,CA,County,30573,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00
309251,2021-04-27,Imperial,California,28148,719.00,6025,CA,County,181215,15532.93,396.77,396.77,15532.93,1,2,12.00,0.00
656780,2021-04-27,Kings,California,22850,246.00,6031,CA,County,152940,14940.50,160.85,160.85,14940.50,11,3,3.00,0.00
153999,2021-04-27,San Bernardino,California,295665,4572.00,6071,CA,County,2180085,13562.09,209.72,209.72,13562.09,3,4,100.00,1.00
2297,2021-04-27,Los Angeles,California,1232742,23802.00,6037,CA,County,10039107,12279.40,237.09,237.09,12279.40,2,5,273.00,3.00
31917,2021-04-27,Riverside,California,298259,4563.00,6065,CA,County,2470546,12072.59,184.70,184.70,12072.59,6,6,258.00,4.00
193617,2021-04-27,Kern,California,108418,1345.00,6029,CA,County,900202,12043.74,149.41,149.41,12043.74,14,7,105.00,0.00
490825,2021-04-27,Merced,California,31489,454.00,6047,CA,County,277680,11340.03,163.50,163.50,11340.03,10,8,37.00,0.00
68869,2021-04-27,Stanislaus,California,61095,1040.00,6099,CA,County,550660,11094.87,188.86,188.86,11094.87,5,9,70.00,1.00
85386,2021-04-27,Tulare,California,49547,834.00,6107,CA,County,466195,10627.96,178.90,178.90,10627.96,7,10,32.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
309251,2021-04-27,Imperial,California,28148,719.00,6025,CA,County,181215,15532.93,396.77,396.77,15532.93,1,2,12.00,0.00
2297,2021-04-27,Los Angeles,California,1232742,23802.00,6037,CA,County,10039107,12279.40,237.09,237.09,12279.40,2,5,273.00,3.00
153999,2021-04-27,San Bernardino,California,295665,4572.00,6071,CA,County,2180085,13562.09,209.72,209.72,13562.09,3,4,100.00,1.00
541608,2021-04-27,Inyo,California,1396,36.00,6027,CA,County,18039,7738.79,199.57,199.57,7738.79,4,26,0.00,0.00
68869,2021-04-27,Stanislaus,California,61095,1040.00,6099,CA,County,550660,11094.87,188.86,188.86,11094.87,5,9,70.00,1.00
31917,2021-04-27,Riverside,California,298259,4563.00,6065,CA,County,2470546,12072.59,184.70,184.70,12072.59,6,6,258.00,4.00
85386,2021-04-27,Tulare,California,49547,834.00,6107,CA,County,466195,10627.96,178.90,178.90,10627.96,7,10,32.00,1.00
54795,2021-04-27,San Joaquin,California,72281,1346.00,6077,CA,County,762148,9483.85,176.61,176.61,9483.85,8,19,294.00,0.00
31083,2021-04-27,Fresno,California,101099,1662.00,6019,CA,County,999101,10119.00,166.35,166.35,10119.00,9,14,38.00,0.00
490825,2021-04-27,Merced,California,31489,454.00,6047,CA,County,277680,11340.03,163.50,163.50,11340.03,10,8,37.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15532.93,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4052,04/23/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,6.00,0.00
4053,04/24/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00
4054,04/25/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00
4055,04/26/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15532.93,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15532.93,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4138,04/23/21,Merced,31400,454.00,11307.98,163.50,163.50,11340.03,10,8,42.00,0.00
4139,04/24/21,Merced,31400,454.00,11307.98,163.50,163.50,11340.03,10,8,0.00,0.00
4140,04/25/21,Merced,31400,454.00,11307.98,163.50,163.50,11340.03,10,8,0.00,0.00
4141,04/26/21,Merced,31452,454.00,11326.71,163.50,163.50,11340.03,10,8,52.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)